In [1]:
from sys import argv
import numpy as np

from thesis_v2.data.prepared.yuanyuan_8k import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished.builder import (gen_maskcnn_polished, load_modules)
from thesis_v2.models.fix_kernel_cnn.builder import (gen_fkcnn, load_modules)

In [2]:
load_modules()

# keeping mean response at 0.5 seems the best. somehow. using batch norm is bad, somehow.
datasets_raw = get_data('a', 200, 100, ('042318', '043018', '051018'), scale=0.5)

datasets_raw = {
    'X_train': datasets_raw[0].astype(np.float32),
    'y_train': datasets_raw[1],
    'X_val': datasets_raw[2].astype(np.float32),
    'y_val': datasets_raw[3],
    'X_test': datasets_raw[4].astype(np.float32),
    'y_test': datasets_raw[5],
}
weight = np.load('filters.npy')
weight = weight[[0,3,5,8,9,11,14,17,18,20,23,25,27,32,36,37,40,44,53,57,58,64,65,74],:,:]
weight = weight.reshape(24,1,9,9)

In [3]:
import torch
import matplotlib.pyplot as plt
import torchvision.utils
def imshow(img, figsize, savefile):
    plt.close('all')
    plt.figure(figsize=figsize)
    npimg = img.numpy()
    img = np.transpose(npimg, (1, 2, 0))
    plt.imshow(img)
    plt.show()
    if savefile is not None:
        imsave(savefile, img)
    
    
def display_one_network(kernels, scale_each=True, normalize=True,figsize=(8,6), nrow=6, savefile=None):
    #kernels = model.first_layer.weight.data.cpu().detach()
    kernels = torch.from_numpy(kernels)
    imshow(torchvision.utils.make_grid(kernels, nrow=nrow, normalize=normalize, scale_each=scale_each),
          figsize=figsize, savefile=savefile)

In [8]:
def gen_cnn_partial(input_size, n):
    return gen_fkcnn(input_size=input_size,
                         num_neuron=n,
                         kernel_size_l1=9, out_channel_l1=24, kernel_size_l23=5, out_channel_l23=24,
                         factored_constraint=None,
                         act_fn='relu',
                         do_init=True,
                         pooling_type='max',
                         pooling_ksize=2,
                         num_layer=2,
                         bn_before_act=True,
                         bn_after_fc=False
                         )
opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=0.01,
                                 smoothness=0.00005,
                                 group=0.0,
                                 first_layer_no_learning=True)
results = []
for seed in range(3):
    results.append(train_one(arch_json_partial=gen_cnn_partial,
                             opt_config_partial=opt_config_partial,
                             datasets=datasets_raw,
                             key=f'debug/FKCNN/v2_code/image-v1/{seed}',
                             init_first_layer=weight,
                             show_every=1000,
                             model_seed=seed,
                             return_model=False)
                   )

print([r['stats_best']['stats']['test']['corr_mean'] for r in results])
print(sum([r['stats_best']['stats']['test']['corr_mean'] for r in results])/3)

num_param 185581
========starting phase 1/3==========
val metric init {'loss': 1.0587507486343384, 'loss_no_reg': 1.0216825008392334, 'corr': None, 'corr_mean': 0.0059945425018668175, 'corr_mean_neg': -0.0059945425018668175, 'corr2_mean': 0.002123050158843398, 'corr2_mean_neg': -0.002123050158843398, 'acc': None}
========starting epoch 0==========
0-0, train loss 0.8839415311813354
train loss 0.8839415311813354
val metric {'loss': 0.8757548928260803, 'loss_no_reg': 0.8446552753448486, 'corr': None, 'corr_mean': 0.021243758499622345, 'corr_mean_neg': -0.021243758499622345, 'corr2_mean': 0.004318195395171642, 'corr2_mean_neg': -0.004318195395171642, 'acc': None}
test metric {'loss': 0.8764614122254508, 'loss_no_reg': 0.8470839858055115, 'corr': None, 'corr_mean': 0.02512964978814125, 'corr_mean_neg': -0.02512964978814125, 'corr2_mean': 0.004088208079338074, 'corr2_mean_neg': -0.004088208079338074, 'acc': None}
========done epoch 0==========
========starting epoch 1000==========
1000-0, t

early stopping after epoch 1100 metric 0.8107060194015503
========end phase 1/3==========
========starting phase 2/3==========
for grp of sz 10, lr from 0.001000 to 0.000333
for grp of sz 1, lr from 0.000000 to 0.000000
val metric init {'loss': 0.8221910476684571, 'loss_no_reg': 0.8107060194015503, 'corr': None, 'corr_mean': 0.533545970916748, 'corr_mean_neg': -0.533545970916748, 'corr2_mean': 0.30461496114730835, 'corr2_mean_neg': -0.30461496114730835, 'acc': None}
========starting epoch 0==========
0-0, train loss 0.8166915774345398
train loss 0.8166915774345398
val metric {'loss': 0.8221339821815491, 'loss_no_reg': 0.8106505870819092, 'corr': None, 'corr_mean': 0.5334783792495728, 'corr_mean_neg': -0.5334783792495728, 'corr2_mean': 0.30456116795539856, 'corr2_mean_neg': -0.30456116795539856, 'acc': None}
test metric {'loss': 0.8234126397541591, 'loss_no_reg': 0.8139877319335938, 'corr': None, 'corr_mean': 0.530394971370697, 'corr_mean_neg': -0.530394971370697, 'corr2_mean': 0.301027

In [ ]:
kernel = res['model'].moduledict['conv0'].weight.detach().numpy()
display_one_network(kernel)